<a href="https://colab.research.google.com/github/SJ-1011/Pill_Data/blob/main/Pill_Classification_Model_Tarining_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install pkbar
!pip install torchbearer

#### 이미지 split


In [ ]:
import cv2
import numpy as np
import os


import os
import shutil
from PIL import Image


class Separate():
    def __init__(self, config):
        '''
        filter_folder_path : filter image save path
        folder_save_path : traing/validation/traing separate image folder save path
        '''

        self.open_path = config['save_path']
        self.save_path = config['separate_path']


    def FolderList(self):
        type_list = []
        folder_list = []

        for (path, folder, files) in os.walk(self.open_path):
            type_list.append(path)
            folder_list.append(folder)

        folders = ','.join(folder_list[0])
        folder_list = folders.split(',')
        type_list.pop(0)

        return folder_list


    def makeSubfolder(self, new_dir_path, folder_list):
        for num in range(len(folder_list)):
            os.makedirs(new_dir_path + folder_list[num])


    def ml_directory(self, folder_list):
        if not os.path.exists(self.save_path + 'training'):
            os.makedirs(self.save_path + 'training')
            self.makeSubfolder(self.save_path + 'training/', folder_list)

        if not os.path.exists(self.save_path + 'testing'):
            os.makedirs(self.save_path + 'testing')
            self.makeSubfolder(self.save_path + 'testing/', folder_list)

        if not os.path.exists(self.save_path + 'validation'):
            os.makedirs(self.save_path + 'validation')
            self.makeSubfolder(self.save_path + 'validation/', folder_list)


    def separate(self, dir_path, x):
        dirname = self.open_path + x
        filenames = os.listdir(dirname)
        i = 0
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)

            with Image.open(full_filename) as image:
                if i % 10 < 7:
                    training_directory = os.path.join(dir_path + 'training/', x)
                    shutil.copyfile(full_filename, os.path.join(training_directory, filename))

                elif i % 10 >= 7 and i % 10 < 8:
                    validation_directory = os.path.join(dir_path + 'testing/', x)
                    shutil.copyfile(full_filename, os.path.join(validation_directory, filename))

                else:
                    testing_directory = os.path.join(dir_path + 'validation/', x)
                    shutil.copyfile(full_filename, os.path.join(testing_directory, filename))
            i = i + 1
            print(f'{filename} finish')


    def separateProcess(self):
        folder_list = self.FolderList()
        self.ml_directory(folder_list)

        for x in folder_list:
            self.separate(self.save_path, x)

import configparser
import sys


class PreprocessingMain():
    def __init__(self):
        self.config_file = '/content/gdrive/MyDrive/preprocessing.ini'

    def main(self, argv):
        # ex) python3 PreProcessing.py circle config.ini
        self.config_file = argv[1]

        shape = argv[0]

        config = configparser.ConfigParser()
        config.read(self.config_file, encoding='UTF-8')

        self.Separate = Separate(config['img_processing'])

        """ Image Separte"""
        self.Separate.separateProcess()

        print('####### finish #######')


if __name__ == '__main__':
    main_class = PreprocessingMain()
    main_class.main(['none', '/content/gdrive/MyDrive/preprocessing.ini'])



#### 이미지 Training


In [ ]:
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# image file truncated error prevention
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

class ResizeAndPad(object):
    def __init__(self, desired_size):
        self.desired_size = desired_size

    def __call__(self, image):
        desired_width, desired_height = self.desired_size
        original_width, original_height = image.size

        # 비율 유지하여 리사이즈
        ratio = min(desired_width / original_width, desired_height / original_height)
        new_width = int(original_width * ratio)
        new_height = int(original_height * ratio)
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)

        # 새 이미지 생성 및 패딩
        new_image = Image.new("RGB", (desired_width, desired_height))
        pad_left = (desired_width - new_width) // 2
        pad_top = (desired_height - new_height) // 2
        new_image.paste(resized_image, (pad_left, pad_top))

        return new_image

class PyTorchData():
    def __init__(self, _dataType, config):
        '''
        input_dim : image size
        data_path : training data path
        batch_size : batch size
        '''

        if _dataType == "data":
            self.m_DataDim = int(config['input_dim'])
        elif _dataType == "image":
            self.m_ImageDim = int(config['input_dim'])
            self.m_DataPath = config['data_path']
            self.m_BatchSize = int(config['batch_size'])


    def ImageTrain(self):
        # 이미지를 200x200으로 변환
        transDatagen = transforms.Compose([ResizeAndPad((self.m_ImageDim, self.m_ImageDim)),
                                           transforms.ToTensor()])

        trainPath = self.m_DataPath + '/training'
        trainFolder = torchvision.datasets.ImageFolder(root = trainPath,
                                                       transform = transDatagen)

        trainLoader = DataLoader(trainFolder,
                                batch_size = self.m_BatchSize,
                                shuffle = True)

        print("Train Class [", trainLoader.dataset.class_to_idx, "]")
        print("Train Numbers [", len(trainLoader.dataset.imgs), "]")
        print("Train Batch Size [", trainLoader.batch_size, "]")

        return trainLoader


    def ImageValidation(self):
        transDatagen = transforms.Compose([ResizeAndPad((self.m_ImageDim, self.m_ImageDim)),
                                           transforms.ToTensor()])

        validationPath = self.m_DataPath + '/validation'
        validationSet = torchvision.datasets.ImageFolder(root = validationPath,
                                                         transform = transDatagen)

        validationLoader = DataLoader(validationSet,
                                      batch_size = self.m_BatchSize,
                                      shuffle = False)

        print("Validation Class [", validationLoader.dataset.class_to_idx, "]")
        print("Validation Numbers [", len(validationLoader.dataset.imgs),"]")
        print("Validation Batch Size [", validationLoader.batch_size,"]")

        return validationLoader


    def ImageTest(self):
        transDatagen = transforms.Compose([ResizeAndPad((self.m_ImageDim, self.m_ImageDim)),
                                           transforms.ToTensor()])

        testDirectory = self.m_DataPath + '/testing'
        testSet = torchvision.datasets.ImageFolder(root = testDirectory,
                                                   transform = transDatagen)

        testLoader = DataLoader(testSet,
                                batch_size = self.m_BatchSize,
                                shuffle = False)

        print("Test Class [", testLoader.dataset.class_to_idx, "]")
        print("Test Numbers [", len(testLoader.dataset.imgs), "]")
        print("Test Batch Size [", testLoader.batch_size,"]")

        return testLoader


import torch
import torch.nn as nn
import pkbar
import numpy as np
import random
import torchvision.models as models


# seed fix
SEED = 3
np.random.seed(SEED)
random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.manual_seed(SEED)


class PillModel(nn.Module):
    # bulid cnn model
    def __init__(self, config):
        super(PillModel, self).__init__()
        '''
        ClassNum : class number
        '''

        self.m_ClassNum = int(config['class_num'])
        # ResNet 모델 초기화
        self.resnet = models.resnet18(pretrained=True)

        # 마지막 fully connected layer의 출력 크기를 클래스 수에 맞게 조정
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, self.m_ClassNum)


    def forward(self, x):
        # ResNet 모델의 forward pass 수행
        x = self.resnet(x)
        return x


import torch.optim as optim
import torchbearer
from torchbearer import Trial
import datetime


class MakeModel():
    def __init__(self,config):
        '''
        learning_rate : learning rate
        epochs : epoch
        save_path : save path
        model_name : model save name
        '''

        self._epoch = int(config['epochs'])
        self._lr = float(config['learning_rate'])
        self._savePath = config['save_path']
        self._modelName = config['model_name']


    def SaveModel(self, _model, optimizer, _trainData, trainLoss):
        nowdate = datetime.datetime.now().strftime('%y%m%d_%H')
        ret = 0
        try:
            torch.save({'model_state_dict': _model.state_dict(),
                        'epoch': self._epoch,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': trainLoss,
                        'label_name':_trainData.dataset.classes},
                        self._savePath + self._modelName + '_PyTorchModel.pt')
            print("model saved [", self._savePath +  self._modelName + "_PyTorchModel.pt ]")

        except PermissionError:
                torch.save(_model, './' + nowdate + '_' + self._modelName + '_PyTorchModel.pt')
                print('model saved [ ./' + nowdate + '_' + self._modelName + '_PyTorchModel.pt ]')

        except IOError as e:
            print("IOError except: ", e.errno)
            ret = 1

        return ret


    def Training(self, _device, _model, _trainData, _valData):
        _model.train()

        optimizer = optim.Adam(_model.parameters(), lr = self._lr)
        criterion = torch.nn.CrossEntropyLoss()
        bestValLoss = float('inf')

        for epoch in range(self._epoch):
            trainLoss = 0.0
            trainSize = 0.0
            trainCorrect = 0.0

            print("Epoch {}/{}".format(epoch + 1, self._epoch))
            progress = pkbar.Kbar(target=len(_trainData), width = 25)

            # train
            for batchIdx, data in enumerate(_trainData):
                images, labels = data
                images, labels = images.to(_device), labels.to(_device)

                optimizer.zero_grad()
                outputs = _model(images)

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                trainLoss = loss.item()

                _, predicted = outputs.max(1)
                trainSize += labels.shape[0]
                trainCorrect += predicted.eq(labels.view_as(predicted)).sum().item()
                trainAccuracy = 100 * trainCorrect / trainSize

                progress.update(batchIdx, values = [("loss: ", trainLoss), ("acc: ", trainAccuracy)])

                del loss
                del outputs

            # validation
            with torch.no_grad():
                valLoss = 0.0
                valSize = 0.0
                valCorrect = 0.0

                for batchIdx, data in enumerate(_valData):
                    images, labels = data
                    images, labels = images.to(_device), labels.to(_device)

                    outputs = _model(images)
                    valLoss = criterion(outputs, labels).item()

                    _, predicted = outputs.max(1)
                    valSize += labels.shape[0]

                    valCorrect += predicted.eq(labels.view_as(predicted)).sum().item()
                    valAccuracy = 100 * valCorrect / valSize

                progress.add(1, values=[("val loss", valLoss), ("val acc", valAccuracy)])

            # if best loss value, save model
            if valLoss < bestValLoss:
                bestValLoss = valLoss
                ret = self.SaveModel(_model, optimizer, _trainData, trainLoss)

        return ret


    # test set model test
    def Testing(self, _device, _model, _testData):
        _model.eval()
        criterion = torch.nn.CrossEntropyLoss()

        testLoss = 0.0
        testSize = 0.0
        testCorrect = 0.0
        all_labels = []
        all_predictions = []

        progress = pkbar.Kbar(target=len(_testData), width=25)

        with torch.no_grad():
            for batchIdx, data in enumerate(_testData):
                images, labels = data
                images, labels = images.to(_device), labels.to(_device)
                outputs = _model(images)

                loss = criterion(outputs, labels).item()
                testLoss += loss

                _, predicted = outputs.data.max(1)
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

                testSize += labels.shape[0]
                testCorrect += predicted.eq(labels.view_as(predicted)).sum().item()
                accuracy = 100 * testCorrect / testSize

                progress.update(batchIdx, values=[("test loss: ", testLoss / (batchIdx + 1)), ("test acc: ", accuracy)])

            testLoss /= len(_testData.dataset)
        progress.add(1)

        # Confusion Matrix
        cm = confusion_matrix(all_labels, all_predictions)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.show()




import torch
import configparser
import torch.nn as nn


class PyTorchMain():
    def __init__(self):
        config = configparser.ConfigParser()
        config.read('/content/gdrive/MyDrive/modeling_config.ini', encoding='UTF-8')

        self.m_Device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f'[Device]: {self.m_Device}')

        self.m_cPytorchModel = PillModel(config['PT_model_info'])

        self.m_cPytorchData = PyTorchData("image", config['PT_model_info'])

        self.m_cMakeModel = MakeModel(config['PT_model_info'])


    def main(self):
        print("\n[ Model ]\n")
        model = self.m_cPytorchModel.to(self.m_Device)
        print(model)

        # load dataset
        print("\n[ Data ]\n")
        trainData = self.m_cPytorchData.ImageTrain()
        valData = self.m_cPytorchData.ImageValidation()
        testData = self.m_cPytorchData.ImageTest()

        # training
        print("\n[ Training ]\n")
        ret = self.m_cMakeModel.Training(_device = self.m_Device,
                                         _model = model,
                                         _trainData = trainData,
                                         _valData = valData)
        if ret == 0 or ret == 1:
            # testing
            print("\n[ Testing ]\n")
            self.m_cMakeModel.Testing(_device = self.m_Device,
                                      _model = model,
                                      _testData = testData)


if __name__ == '__main__':
    mainClass = PyTorchMain()
    mainClass.main()


[Device]: cuda

[ Model ]

PillModel(
  (m_Conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (m_Pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (m_Conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (m_Pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (m_Conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (m_Pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (m_Linear4): Linear(in_features=246016, out_features=256, bias=True)
  (m_Drop4): Dropout2d(p=0.5, inplace=False)
  (m_Linear5): Linear(in_features=256, out_features=20, bias=True)
  (m_Relu): ReLU()
)

[ Data ]

Train Class [ {'196000001': 0, '196500004': 1, '199502575': 2, '200008571': 3, '200402485': 4, '200402928': 5, '200404719': 6, '200410999': 7, '200600116': 8, '200702709': 9, '200702970': 10, '200806299': 11, '201207004': 12, '201403243': 13, '201

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


 7/88 [>........................] - ETA: 1:11:52 - loss: : 3.0436 - acc: : 6.9417

KeyboardInterrupt: 

#### Predict

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random
from PIL import Image
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

class ResizeAndPad(object):
    def __init__(self, desired_size):
        self.desired_size = desired_size

    def __call__(self, image):
        desired_width, desired_height = self.desired_size
        original_width, original_height = image.size

        # 비율 유지하여 리사이즈
        ratio = min(desired_width / original_width, desired_height / original_height)
        new_width = int(original_width * ratio)
        new_height = int(original_height * ratio)
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)

        # 새 이미지 생성 및 패딩
        new_image = Image.new("RGB", (desired_width, desired_height))
        pad_left = (desired_width - new_width) // 2
        pad_top = (desired_height - new_height) // 2
        new_image.paste(resized_image, (pad_left, pad_top))

        return new_image

class PillModel(nn.Module):
    # bulid cnn model
    def __init__(self):
        super(PillModel, self).__init__()

        # 클래스 개수 정의
        self.m_ClassNum = 20
        # ResNet 모델 초기화
        self.resnet = models.resnet18(pretrained=True)

        # 마지막 fully connected layer의 출력 크기를 클래스 수에 맞게 조정
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, self.m_ClassNum)


    def forward(self, x):
        # ResNet 모델의 forward pass 수행
        x = self.resnet(x)
        return x

# 이미지 전처리 함수
def preprocess_image(image_path):
    transform = transforms.Compose([ResizeAndPad((500, 500)),
                                           transforms.ToTensor()])
    image = Image.open(image_path).convert("RGB")  # 이미지를 RGB 형식으로 열기
    image = transform(image).unsqueeze(0)         # 배치 차원 추가
    return image

# 모델 로드
model = PillModel()
checkpoint = torch.load("/content/gdrive/MyDrive/sample_model/20_class_model_random.pt", map_location=torch.device('cpu'))  # 모델 파일 로드
model.load_state_dict(checkpoint['model_state_dict'])  # 모델의 상태 사전 로드
model.eval()

# 클래스 이름 목록
class_names = checkpoint['label_name']

# 이미지 분류 함수
def classify_image(image, model, class_names):


    # 모델 추론
    with torch.no_grad():
        output = model(image)

    # 확률로 변환
    probabilities = torch.softmax(output, dim=1)

    print(probabilities)
    # 클래스 이름과 해당 확률 출력
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: {probabilities[0][i].item():.2f}")

    # 클래스 예측
    _, predicted_class = torch.max(probabilities, 1)

    # 클래스 이름 가져오기
    class_name = class_names[predicted_class.item()]

    return class_name, probabilities[0][predicted_class].item()

# 이미지 분류
image_path = "/content/gdrive/MyDrive/test_200600116.jpg"  # 이미지 파일 경로

# 이미지 전처리
image = preprocess_image(image_path)

predicted_class, confidence = classify_image(image, model, class_names)

# 결과 출력
print("예측된 클래스:", predicted_class)
print("확신도:", confidence)

tensor([[1.8850e-03, 1.7215e-05, 8.8273e-06, 3.2764e-05, 5.0358e-05, 2.0530e-05,
         2.7430e-05, 5.7827e-04, 9.9700e-01, 8.9350e-05, 6.2422e-06, 2.2250e-06,
         2.7124e-05, 9.3597e-06, 6.9638e-06, 1.3619e-05, 1.0528e-05, 7.3349e-06,
         1.5508e-05, 1.8645e-04]])
196000001: 0.00
196500004: 0.00
199502575: 0.00
200008571: 0.00
200402485: 0.00
200402928: 0.00
200404719: 0.00
200410999: 0.00
200600116: 1.00
200702709: 0.00
200702970: 0.00
200806299: 0.00
201207004: 0.00
201403243: 0.00
201503295: 0.00
201701435: 0.00
201705221: 0.00
201901014: 0.00
201906965: 0.00
201907607: 0.00
예측된 클래스: 200600116
확신도: 0.9970049262046814
